In [ ]:
%%capture
%run ../ImportsConstantsSparkInit.ipynb
sc.setJobGroup("DA_Temperature", "DA_Temperature")

In [ ]:
%%time
pipeline = "[{$match:{'Temperature-F':{$gte:-20, $lte:120}}},{ $bucket : { groupBy : '$Temperature-F', boundaries:[-30,-20,-10,0,10,20,30,40,50,60,70,80,90,100,110,120,130], output:{count:{$sum:1}}}}, {$sort:{_id:-1}}]"
df = spark.read.format("mongo").option("collection", "TrafficData").option("pipeline", pipeline).load()
df = df.groupby("_id").sum("count")
df = df.withColumnRenamed("_id","Temperature").withColumnRenamed("sum(count)","AccidentCount")
df = df.orderBy("Temperature")
#print("Number of Documents: " + str(df.count()))
#df.agg({'AccidentCount': 'sum'}).show()
#df.show(49)

In [ ]:
df.write.format("mongo").option("collection", "AccidentCountByTemperature").mode("overwrite").save()

In [ ]:
print("Data Aggregation for Temperature Data Finished")